<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=45dd896a67073eb1363bfc3c6662fafbe315ca811534513a1269243002edc0a0
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-01 10:10:53
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  1.57 L
Current:  1.39 C
-------------------
Today PnL: 25.18 K (0.18%)
Current PnL: -22.57 L (-14.98%)
CY Booked + Current PnL: -11.13 L (-7.38%)
-------------------
Total profit:  1.10 L
Total loss:  -23.67 L
-------------------
Total Booked + Current PnL: 15.85 L (12.82%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.23%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.28 C
Est FTT PnL: 88.94 L (63.97%)
Deployed:  1.24 C
Current:  1.39 C
CAGR/XIRR %: 7.14%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.53,113367.0,12863.0,16245.0,0.01,12.80,14.33,28.96,79.0,0.79,0.82,51.11,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.60,37.0,M-SC,2.45,83368.0,-17409.0,17499.0,0.23,-17.27,20.99,0.09,245.0,-0.99,0.61,9.53,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,7.49,41.0,H-MC,3.36,185440.0,20770.0,22568.0,-2.17,12.61,12.17,26.31,89.0,0.92,1.35,21.60,X40N,NTT,AC
57,RELIANCE,1533.00,-14.41,44.0,H-LC,3.71,214313.0,3947.0,24839.0,0.72,1.88,11.59,13.68,37.0,0.16,1.56,18.71,XY25,NTT,REFINERIES
42,ITC,452.00,-39.19,41.0,H-LC,2.09,195918.0,-4220.0,25117.0,-0.22,-2.11,12.82,10.44,4.0,-0.17,1.43,3.34,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,64.47,75.0,H-SC,1.60,151817.0,-27254.0,92988.0,12.00,-15.22,61.25,36.71,135.0,-0.29,1.10,50.63,XR,NTT,BANKS
64,SHALBY,327.00,1242.90,69.0,M-SC,14.27,182606.0,1463.0,44008.0,6.62,0.81,24.10,25.10,235.0,0.03,1.33,46.34,XY24,NTT,HEALTHCARE
26,GREENPANEL,537.00,213.31,50.0,M-SC,1.18,139885.0,-43193.0,122176.0,3.86,-23.59,87.34,43.14,230.0,-0.35,1.02,31.58,XY24,NTT,MISC
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,40.35,79680.0,-33869.0,73919.0,2.65,-29.83,92.77,35.27,268.0,-0.46,0.58,95.02,XR,NTT,HOTELS
52,PGHH,17973.08,-27.37,64.0,H-MC,3.22,210060.0,9240.0,59531.0,2.61,4.60,28.34,34.25,80.0,0.16,1.53,10.04,X40,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-46.62,19.0,H-SC,14.33,80920.0,-21580.0,62179.0,-5.38,-21.05,76.84,39.61,114.0,-0.35,0.59,0.00,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-48.11,23.0,M-SC,1.18,87780.0,-3718.0,82469.0,-5.05,-4.06,93.95,86.07,223.0,-0.05,0.64,27.47,XR,NTT,DURABLES
3,ALKYLAMINE,4546.37,-11.74,29.0,H-SC,10.70,82220.0,-18743.0,113274.0,-2.95,-18.56,137.77,93.63,148.0,-0.17,0.60,22.47,SR,ATH,CHEMICALS
81,VAIBHAVGBL,521.00,47.11,43.0,H-SC,5.81,127481.0,-55294.0,166363.0,-2.77,-30.25,130.50,60.77,125.0,-0.33,0.93,14.18,XR,NTT,JEWELLERY
72,TATAELXSI,9161.00,-27.24,28.0,H-MC,6.39,98506.0,-33329.0,75554.0,-2.20,-25.28,76.70,32.03,98.0,-0.44,0.72,10.33,OX40N,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1242.90,69.0,M-SC,14.27,182606.0,1463.0,44008.0,6.62,0.81,24.10,25.10,235.0,0.03,1.33,46.34,XY24,NTT,HEALTHCARE
37,INDIAMART,4850.92,-55.39,24.0,H-SC,9.97,124370.0,1034.0,132728.0,-0.85,0.84,106.72,108.45,119.0,0.01,0.90,23.38,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,97.20,57.0,H-SC,11.23,124902.0,-17577.0,37096.0,-0.19,-12.34,29.70,13.70,163.0,-0.47,0.91,47.42,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.60,37.0,M-SC,2.45,83368.0,-17409.0,17499.0,0.23,-17.27,20.99,0.09,245.0,-0.99,0.61,9.53,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,51.0,H-SC,1.54,221130.0,-48537.0,84870.0,-0.36,-18.00,38.38,13.47,138.0,-0.57,1.61,11.84,XY24,NTT,PAINTS
18,CERA,9475.0,-22.91,35.0,H-SC,1.61,140622.0,-35281.0,77300.0,-0.72,-20.06,54.97,23.89,149.0,-0.46,1.02,21.45,OX40N,NTT,CERAMICS
66,SIS,528.0,2090.16,42.0,H-SC,4.03,88383.0,-22649.0,46260.0,0.41,-20.40,52.34,21.26,156.0,-0.49,0.64,19.41,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1242.90,69.0,M-SC,14.27,182606.0,1463.0,44008.0,6.62,0.81,24.10,25.10,235.0,0.03,1.33,46.34,XY24,NTT,HEALTHCARE
60,SAIL,228.00,47.55,60.0,M-MC,11.52,232728.0,7766.0,159884.0,0.50,3.45,68.70,74.53,192.0,0.05,1.69,36.82,XY24,BTT,STEEL
32,HINDZINC,730.22,28.52,65.0,M-LC,3.87,209779.0,4703.0,107868.0,-0.04,2.29,51.42,54.89,52.0,0.04,1.53,26.94,X5K,ATH,METALS
14,BLUESTARCO,2080.00,7.49,41.0,H-MC,3.36,185440.0,20770.0,22568.0,-2.17,12.61,12.17,26.31,89.0,0.92,1.35,21.60,X40N,NTT,AC
84,VOLTAS,1530.00,-3.46,42.0,H-MC,2.83,202560.0,10818.0,26940.0,0.76,5.64,13.30,19.69,99.0,0.40,1.47,12.82,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.53,113367.0,12863.0,16245.0,0.01,12.80,14.33,28.96,79.0,0.79,0.82,51.11,MH,ATH,METALS
11,BANKINDIA,190.00,-26.71,66.0,H-MC,9.41,190404.0,10596.0,101809.0,0.32,5.89,53.47,62.52,88.0,0.10,1.39,41.99,XR,NTT,BANKS
32,HINDZINC,730.22,28.52,65.0,M-LC,3.87,209779.0,4703.0,107868.0,-0.04,2.29,51.42,54.89,52.0,0.04,1.53,26.94,X5K,ATH,METALS
37,INDIAMART,4850.92,-55.39,24.0,H-SC,9.97,124370.0,1034.0,132728.0,-0.85,0.84,106.72,108.45,119.0,0.01,0.90,23.38,AR,ATH,MISC
86,WIPRO,420.00,-15.61,37.0,M-LC,5.82,147913.0,-3032.0,112488.0,-0.33,-2.01,76.05,72.51,53.0,-0.03,1.08,3.89,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.94,27.0,H-LC,5.82,204477.0,-47291.0,100500.0,-1.12,-18.78,49.15,21.13,27.0,-0.47,1.49,9.99,X40,ATH,PAINTS
44,JIOFIN,387.00,-6.90,27.0,H-LC,12.85,252883.0,-14482.0,79936.0,0.29,-5.42,31.61,24.48,48.0,-0.18,1.84,46.58,XY24,BTT,FINANCE
74,TCS,4476.75,-32.22,27.0,H-LC,12.44,270391.0,-75565.0,150419.0,-0.41,-21.84,55.63,21.64,1.0,-0.50,1.97,0.00,X40,ATH,IT
30,HCLTECH,1943.91,-4.52,34.0,H-LC,10.25,217885.0,-24039.0,87307.0,0.20,-9.94,40.07,26.15,8.0,-0.28,1.59,2.96,X40,ATH,IT
83,VBL,671.64,-20.95,34.0,H-LC,9.23,284803.0,-31039.0,143712.0,0.61,-9.83,50.46,35.67,5.0,-0.22,2.07,2.67,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-8.38,39.0,H-LC,1.56,227115.0,-19036.0,98250.0,-2.15,-7.73,43.26,32.18,16.0,-0.19,1.65,25.52,X200,ATH,IT
65,SIEMENS,4671.5,-2.61,45.0,H-LC,1.97,156635.0,-29460.0,76939.0,0.11,-15.83,49.12,25.51,15.0,-0.38,1.14,15.43,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.19,41.0,H-LC,2.09,195918.0,-4220.0,25117.0,-0.22,-2.11,12.82,10.44,4.0,-0.17,1.43,3.34,X40,NTT,FMCG
57,RELIANCE,1533.0,-14.41,44.0,H-LC,3.71,214313.0,3947.0,24839.0,0.72,1.88,11.59,13.68,37.0,0.16,1.56,18.71,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-12.81,38.0,H-LC,3.73,268398.0,2972.0,53814.0,-0.51,1.12,20.05,21.40,11.0,0.06,1.95,8.53,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-32.22,27.0,H-LC,12.44,270391.0,-75565.0,150419.0,-0.41,-21.84,55.63,21.64,1.0,-0.50,1.97,0.00,X40,ATH,IT
83,VBL,671.64,-20.95,34.0,H-LC,9.23,284803.0,-31039.0,143712.0,0.61,-9.83,50.46,35.67,5.0,-0.22,2.07,2.67,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.52,34.0,H-LC,10.25,217885.0,-24039.0,87307.0,0.20,-9.94,40.07,26.15,8.0,-0.28,1.59,2.96,X40,ATH,IT
42,ITC,452.00,-39.19,41.0,H-LC,2.09,195918.0,-4220.0,25117.0,-0.22,-2.11,12.82,10.44,4.0,-0.17,1.43,3.34,X40,NTT,FMCG
40,INFY,2275.00,-21.75,36.0,H-LC,8.21,304569.0,-8319.0,180000.0,-0.83,-2.66,59.10,54.87,3.0,-0.05,2.22,3.81,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.61,45.0,H-LC,1.97,156635.0,-29460.0,76939.0,0.11,-15.83,49.12,25.51,15.0,-0.38,1.14,15.43,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,41.0,H-LC,2.09,195918.0,-4220.0,25117.0,-0.22,-2.11,12.82,10.44,4.0,-0.17,1.43,3.34,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.94,27.0,H-LC,5.82,204477.0,-47291.0,100500.0,-1.12,-18.78,49.15,21.13,27.0,-0.47,1.49,9.99,X40,ATH,PAINTS
19,CIPLA,1795.00,-22.87,39.0,H-LC,4.00,205788.0,1288.0,40129.0,0.95,0.63,19.50,20.25,10.0,0.03,1.50,9.42,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-14.41,44.0,H-LC,3.71,214313.0,3947.0,24839.0,0.72,1.88,11.59,13.68,37.0,0.16,1.56,18.71,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-32.22,27.0,H-LC,12.44,270391.0,-75565.0,150419.0,-0.41,-21.84,55.63,21.64,1.0,-0.50,1.97,0.00,X40,ATH,IT
40,INFY,2275.00,-21.75,36.0,H-LC,8.21,304569.0,-8319.0,180000.0,-0.83,-2.66,59.10,54.87,3.0,-0.05,2.22,3.81,X40,BTT,IT
42,ITC,452.00,-39.19,41.0,H-LC,2.09,195918.0,-4220.0,25117.0,-0.22,-2.11,12.82,10.44,4.0,-0.17,1.43,3.34,X40,NTT,FMCG
83,VBL,671.64,-20.95,34.0,H-LC,9.23,284803.0,-31039.0,143712.0,0.61,-9.83,50.46,35.67,5.0,-0.22,2.07,2.67,X40N,ATH,FMCG
1,ABB,7934.00,-40.74,42.0,H-LC,7.72,247584.0,-14035.0,133250.0,-0.30,-5.36,53.82,45.57,7.0,-0.11,1.80,5.35,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,40.35,79680.0,-33869.0,73919.0,2.65,-29.83,92.77,35.27,268.0,-0.46,0.58,95.02,XR,NTT,HOTELS
49,MASFIN,398.61,-20.05,38.0,H-SC,12.34,90915.0,-7065.0,28665.0,0.13,-7.21,31.53,22.05,152.0,-0.25,0.66,32.15,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.53,113367.0,12863.0,16245.0,0.01,12.80,14.33,28.96,79.0,0.79,0.82,51.11,MH,ATH,METALS
78,UJJIVANSFB,60.00,97.20,57.0,H-SC,11.23,124902.0,-17577.0,37096.0,-0.19,-12.34,29.70,13.70,163.0,-0.47,0.91,47.42,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-185.23,79.0,M-SC,8.16,147087.0,-2133.0,146307.0,1.60,-1.43,99.47,96.62,208.0,-0.01,1.07,59.18,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1242.90,69.0,M-SC,14.27,182606.0,1463.0,44008.0,6.62,0.81,24.10,25.10,235.0,0.03,1.33,46.34,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-185.23,79.0,M-SC,8.16,147087.0,-2133.0,146307.0,1.60,-1.43,99.47,96.62,208.0,-0.01,1.07,59.18,XY25,NTT,FINANCE
69,SURYODAY,240.00,64.47,75.0,H-SC,1.60,151817.0,-27254.0,92988.0,12.00,-15.22,61.25,36.71,135.0,-0.29,1.10,50.63,XR,NTT,BANKS
60,SAIL,228.00,47.55,60.0,M-MC,11.52,232728.0,7766.0,159884.0,0.50,3.45,68.70,74.53,192.0,0.05,1.69,36.82,XY24,BTT,STEEL
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.53,113367.0,12863.0,16245.0,0.01,12.80,14.33,28.96,79.0,0.79,0.82,51.11,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.11
1,25,43.47
2,50,74.92


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.10
LC    32.36
MC    23.55
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.39
X40      14.62
X40N     11.34
XR       11.15
XY25     10.55
AR        8.15
OX40N     7.83
X200      1.65
MH        1.62
X5K       1.53
SR        1.18
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.15
H-LC    25.71
H-MC    20.58
M-SC    13.49
M-LC     5.59
M-MC     2.67
L-SC     1.46
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.95,-7.01,42.34
IT,12.57,-21.88,88.34
FINANCE,10.45,-17.48,70.85
MISC,6.94,-20.32,87.21
BANKS,6.45,-12.47,71.63
PAINTS,5.57,-20.53,38.49
ELECTRICAL,5.43,-10.30,49.00
INSURANCE,3.77,-6.74,46.64
AC,3.53,0.17,16.55


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3202772.0,22
XR,1377839.0,14
AR,1212062.0,9
X40,842699.0,10
X40N,646089.0,9
OX40N,570610.0,10
XY25,486566.0,7
SR,259503.0,2
X5K,107868.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3785002.0,29
M-SC,1489893.0,17
H-LC,1287314.0,15
H-MC,1276718.0,15
M-LC,383025.0,4
M-MC,310862.0,2
L-SC,261169.0,3
L-MC,60917.0,1
L-LC,39204.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1240914.0      6
           AR         831106.0      5
M-SC       XY24       821502.0      7
H-SC       XR         799127.0      7
H-MC       XY24       586889.0      4
H-LC       X40        543343.0      5
H-SC       X40N       318178.0      4
M-MC       XY24       310862.0      2
H-SC       OX40N      262573.0      4
           SR         259503.0      2
H-LC       X40N       236793.0      3
H-MC       X40        222851.0      4
H-LC       AR         210189.0      2
H-MC       XY25       182252.0      2
L-SC       XR         172174.0      2
M-SC       AR         170767.0      2
M-LC       XY24       162669.0      2
M-SC       XY25       146307.0      1
           OX40N      143488.0      4
           XR         131324.0      2
H-LC       XY25       118803.0      3
M-LC       XR         112488.0      1
           X5K        107868.0      1
H-MC       XR         101809.0      1
H-LC       X200        98250.0      1
H-MC       X40N        91118.0      2
L-SC       OX40N       88995.0      1
H-LC       XY24        79936.0      1
M-SC       X40         76505.0      1
H-MC       OX40N       75554.0      1
H-SC       MH          73601.0      1
L-MC       XR          60917.0      1
L-LC       XY25        39204.0      1
H-MC       MH          16245.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
